In [ ]:
using Distributions
using LinearAlgebra
using Plots
using Test
using Sobol
using Optim
using Profile
using PProf
using Random

In [ ]:
include("../rollout.jl")
include("../testfns.jl")

### Psuedo-code for Rollout Bayesian Optimization
1. Generate low-discrepancy sequence for Quasi-Monte Carlo
2. Gather initial samples/experimental data
3. Construct the ground truth surrogate model
4. Setup hyperparameters for stochastic gradient descent
5. While budget has not been exhausted
<ol>
    <li>
        Construct a batch of samples for stochastic gradient descent. For each sample
        <ol>
            <li>Create a copy of the ground truth surrogate at the sample location and the pairwise perturbed surrogate.</li>
            <li style="color: #f66">Initialize our trajectory struct with the fantasized surrogate and fantisized perturbed surrogate and fantasy start location.</li>
            <li>Perform rollout on the trajectory for $r$ steps $M_0$ times for Quasi-Monte Carlo integration.</li>
            <li>Update values for $\alpha$ and $\nabla\alpha$</li>
        </ol>
    </li>
    <li>Once SGD has converged, update sample location using update rule</li>
    <li>Save location and value at location for each sample in batch.</li>
    <li>Select the best sample from the batch and sample original process at new sample location.</li>
    <li>Update surrogate model with value found at new sample location.</li>
    <li>Repeat until budget is exhausted.</li>
</ol>

### Issues
- Use control variates to see how they affect the rollout acquisition functions

#### Probability of Improvement
The probability of improvement (POI) is defined as follows:

$$
POI(x) = \Phi\left( \frac{\mu(x) - f^+ - \xi}{\sigma(x)} \right)
$$

where $f^+$ denotes the best value (maximum) known, $\mu(x)$ is the predictive mean at x, $\sigma(x)$ is the predictive variance, $\xi$ is our exploration parameter, and $\Phi$ is the standard normal cumulative distribution function.
<hr>

#### Expected Improvement
The expected improvement (EI) is defined as follows:

$$
EI(x) = (\mu(x) - f^+ - \xi)\Phi\left( \frac{\mu(x) - f^+ - \xi}{\sigma(x)} \right) +
        \sigma(x)\phi\left(\frac{\mu(x) - f^+ - \xi}{\sigma(x)}\right)
$$

where $\phi$ is the standard normal probability density function.
<hr>

In [ ]:
function ei(μ, σ, fbest)
    z = (fbest - μ) / σ
    Φz = Distributions.normcdf(z)
    ϕz = Distributions.normpdf(z)
    return σ*(z*Φz + ϕz)
end

function poi(μ, σ, fbest)
    z = (fbest - μ) / σ
    Φz = Distributions.normcdf(z)
    return Φz
end

In [ ]:
# Global parameters
MAX_SGD_ITERS = 100
BATCH_SIZE = 5
HORIZON = 4
MC_SAMPLES = 2
BUDGET = 1;

### 1. Generate low-discrepancy sequence for Quasi-Monte Carlo

In [ ]:
# Setup toy problem
testfn = TestFunction(
    1, [0. 1.], [.5],
    x -> 0. + 1e-6*randn(),
    ∇x -> [0. + 1e-6*randn()]
)
lbs, ubs = testfn.bounds[:,1], testfn.bounds[:,2]

# Setup low discrepancy random number stream
lds_rns = gen_low_discrepancy_sequence(MC_SAMPLES, testfn.dim, HORIZON+1);
rns = randn(MC_SAMPLES, testfn.dim+1, HORIZON+1);

In [ ]:
tplot(testfn)

### 2. Gather initial samples/experimental data

In [ ]:
# Gather initial samples/experimental data
N, θ = 1, [.25]
X = [.15, .85]
X = reshape(X, 1, length(X))
# ψ = kernel_matern52(θ);
ψ = kernel_scale(kernel_matern52, [1., θ...]);

### 3. Construct the ground truth surrogate model

In [ ]:
sur = fit_surrogate(ψ, X, testfn.f);
# θ, sur = optimize_hypers(ψ.θ, kernel_matern52, sur.X, testfn.f);

In [ ]:
domain = filter(x -> !(x in X), lbs[1]:.01:ubs[1])
scatter(sur.X', sur.y)
plot!(domain, [sur([x]).μ for x in domain],
    ribbons=2*[sur([x]).σ for x in domain], label="μ ± 2σ")
plot!(domain, [sur([x]).EI for x in domain], label="EI")

### 4. Setup hyperparameters for stochastic gradient descent

In [ ]:
λ = 1e-1; # Vanilla stochastic gradient descent

### 5. While budget has not been exhausted
Each location in our minibatch is going to be our $x^0$ that serves as our deterministic start location. Then, we perform rollout from that point forward, computing several sample trajectories to then be averaged.

In [ ]:
eis, ∇eis = [], []
T = nothing
for random_number_stream in [lds_rns, rns]
    rollout_ei = [0 0] # Sample mean and variance tuples
    ∇rollout_ei = [0 0] # Sample mean and variance tuples
    
    println("Total '|'s => $(length(domain))")
    for x0 in domain
        print("|$x0")
        # Grab each input location and convert to a column vector
        x0 = [x0]

        αxs, ∇αxs = [], []
        # Monte-carlo integrate trajectory for x0
        for sample in 1:MC_SAMPLES
            # Make a copy of our surrogate to pass to the trajectory struct
            # for fantasized computations
            fsur = Base.deepcopy(sur)
            fantasy_ndx = size(fsur.X, 2) + 1

            # Rollout trajectory
            T = Trajectory(fsur, x0, fantasy_ndx; h=HORIZON)
            rollout!(T, lbs, ubs; rnstream=random_number_stream[sample,:,:])

            # Evaluate rolled out trajectory
            push!(αxs, α(T))
            push!(∇αxs, first(∇α(T)))
        end # endfor sample

        # Average trajectories
        μx = sum(αxs) / length(αxs)
        ∇μx = sum(∇αxs) / length(αxs)
        σx = sum((αxs .- μx) .^ 2) / (MC_SAMPLES-1)
        ∇σx = sum((∇αxs .- ∇μx) .^ 2) / (MC_SAMPLES-1)

        # Update history
        sx = sur(x0)
        μx += ei(sx.μ, sx.σ, minimum(sur.y)) + poi(sx.μ, sx.σ, minimum(sur.y))
        rollout_ei = vcat(rollout_ei, [μx σx])
        ∇rollout_ei = vcat(∇rollout_ei, [∇μx ∇σx])
    end # endfor x0
    rollout_ei = rollout_ei[2:end, :]
    ∇rollout_ei = ∇rollout_ei[2:end, :];
    
    push!(eis, rollout_ei)
    push!(∇eis, ∇rollout_ei)
end

In [ ]:
# xnext = [0.5]
# sxnext = T.s(xnext)
# -sxnext.HEI \ T.δs(sxnext).∇EI
T.s.K

In [ ]:
function signchange_indices(v::Vector)
    indices = [1]
    for i = 2:length(v)
        if sign(v[i]) != sign(v[i-1])
            push!(indices, i)
        end
    end
    return indices
end

In [ ]:
plot_domain = range(lbs[1], ubs[1], length=length(eis[2][:, 2]))
signchange_mc = signchange_indices(∇eis[2][:, 1])
plot(
    plot_domain,
    eis[2][:, 1],
    # ribbons=sqrt.(eis[2][:, 2]),
    label="EI(h=$HORIZON)",
    linestyle=:dash,
)
vline!([plot_domain[loc] for loc in signchange_mc], label="Sign Change")

In [ ]:
signchange_lds = signchange_indices(∇eis[1][:, 1])
plot(
    plot_domain,
    eis[1][:, 1],
    # ribbons=sqrt.(eis[1][:, 2]),
    label="LDS EI(h=$HORIZON)",
    linestyle=:dash,
)
vline!([plot_domain[loc] for loc in signchange_lds], label="Sign Change")

In [ ]:
∇ylims = (minimum(∇eis[2][:, 1]) - 2*maximum(∇eis[2][:, 1]),
    maximum(∇eis[2][:, 1]) + 2*maximum(∇eis[2][:, 1]))
∇ylims_lds = (minimum(∇eis[1][:, 1]) - 2*maximum(∇eis[1][:, 1]),
    maximum(∇eis[1][:, 1]) + 2*maximum(∇eis[1][:, 1]))
∇ylims

In [ ]:
plot(
    plot_domain,
    ∇eis[2][:, 1],
    # ribbons=sqrt.(∇eis[2][:, 2]),
    label="EI(h=$HORIZON)",
    linestyle=:dash,
    ylims=∇ylims
)
vline!([plot_domain[loc] for loc in signchange_mc], label="Sign Change")

In [ ]:
plot(
    plot_domain,
    ∇eis[1][:, 1],
    # ribbons=sqrt.(∇eis[1][:, 2]),
    label="LDS EI(h=$HORIZON)",
    linestyle=:dash,
    ylims=∇ylims
)
vline!([plot_domain[loc] for loc in signchange_lds], label="Sign Change")